<a href="https://colab.research.google.com/github/NoranMorad7/BCI-Assistive-Motion/blob/main/Signal_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

base_path = '/content/drive/MyDrive/Colab Notebooks/files'

# List files
files = os.listdir(base_path)
print(files)


['64_channel_sharbrough.pdf', 'ANNOTATORS', '64_channel_sharbrough-old.png', 'RECORDS', 'SHA256SUMS.txt', '64_channel_sharbrough.png', 'wfdbcal', 'S104', 'S107', 'S109', 'S106', 'S108', 'S101', 'S105', 'S102', 'S103', 'S100', 'S096', 'S090', 'S098', 'S097', 'S095', 'S094', 'S093', 'S099', 'S091', 'S092', 'S083', 'S080', 'S085', 'S086', 'S081', 'S082', 'S088', 'S089', 'S087', 'S084', 'S078', 'S075', 'S074', 'S070', 'S071', 'S072', 'S076', 'S079', 'S077', 'S073', 'S065', 'S066', 'S060', 'S068', 'S067', 'S069', 'S063', 'S061', 'S064', 'S062', 'S051', 'S055', 'S059', 'S057', 'S052', 'S050', 'S053', 'S058', 'S056', 'S054', 'S048', 'S043', 'S044', 'S045', 'S047', 'S040', 'S046', 'S049', 'S042', 'S041', 'S036', 'S037', 'S031', 'S039', 'S034', 'S030', 'S033', 'S032', 'S035', 'S038', 'S025', 'S027', 'S021', 'S023', 'S026', 'S029', 'S020', 'S028', 'S024', 'S022', 'S013', 'S016', 'S010', 'S018', 'S017', 'S011', 'S014', 'S015', 'S019', 'S012', 'S004', 'S003', 'S006', 'S008', 'S007', 'S009', 'S002'

In [ ]:
!pip install mne

import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from scipy import signal
from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
import os
import numpy as np
import mne

base_path = '/content/drive/MyDrive/Colab Notebooks/files'
target_len = 1600
# Create 3 lists for eeg signals, frequances, and labels "rest$ task"
eeg_trials = []
labels = []
fs_list = []

# Select folders that start with "S" and print first 90 files
subject_folders = [f for f in sorted(os.listdir(base_path)) if f.startswith('S') and os.path.isdir(os.path.join(base_path, f))]
print("✅ Found subject folders:", subject_folders[:90])

# Collect .EDF files for each folder
for subj in subject_folders[:90]:
    subj_path = os.path.join(base_path, subj)
    edf_files = [f for f in os.listdir(subj_path) if f.endswith('.edf')]

    print(f"\n📂 Reading from {subj}: {len(edf_files)} EDF files")

# Identify recording num to know Task or Rest
    for fname in edf_files:
        edf_path = os.path.join(subj_path, fname)

        try:
            rec_id = fname.split('R')[-1].split('.')[0]
            rec_num = int(rec_id)
        except:
            continue


        if rec_num in [1, 2]:
            label = 0  # rest
        elif 3 <= rec_num <= 14:
            label = 1  # task
        else:
            continue

# Read the files on only channel1
        try:
            raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)
            sig = raw.get_data()[0]
            fs = raw.info['sfreq']

# To ensure all signals have the same length 1600
            if len(sig) >= target_len:
                sig_fixed = sig[:target_len]
            else:
                sig_fixed = np.pad(sig, (0, target_len - len(sig)), mode='constant')

            eeg_trials.append(sig_fixed)
            fs_list.append(fs)
            labels.append(label)

        except Exception as e:
            print(f"❌ Error reading {fname}: {e}")

print("\n🧠 Trials:", len(eeg_trials), "| fs_list:", len(fs_list), "| Labels:", len(labels))

✅ Found subject folders: ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026', 'S027', 'S028', 'S029', 'S030', 'S031', 'S032', 'S033', 'S034', 'S035', 'S036', 'S037', 'S038', 'S039', 'S040', 'S041', 'S042', 'S043', 'S044', 'S045', 'S046', 'S047', 'S048', 'S049', 'S050', 'S051', 'S052', 'S053', 'S054', 'S055', 'S056', 'S057', 'S058', 'S059', 'S060', 'S061', 'S062', 'S063', 'S064', 'S065', 'S066', 'S067', 'S068', 'S069', 'S070', 'S071', 'S072', 'S073', 'S074', 'S075', 'S076', 'S077', 'S078', 'S079', 'S080', 'S081', 'S082', 'S083', 'S084', 'S085', 'S086', 'S087', 'S088', 'S089', 'S090']

📂 Reading from S001: 14 EDF files

📂 Reading from S002: 14 EDF files

📂 Reading from S003: 14 EDF files

📂 Reading from S004: 14 EDF files

📂 Reading from S005: 14 EDF files

📂 Reading from S006: 14 EDF files

📂 Reading from S007: 14 EDF files

📂 Readin

In [ ]:
# To filter noise and remove useless signals
def bandpass_iir(data, fs, low=4, high=50, order=4):
    nyq = fs / 2
    b, a = signal.butter(order, [low / nyq, high / nyq], btype='band')
    return signal.filtfilt(b, a, data)

# Calculate the power to know values of theta w alph, each one represent different activity in brain
def compute_band_power(x, fs, band):
    freqs = np.fft.fftfreq(len(x), 1/fs)
    fftv = np.abs(fft(x))**2
    mask = (freqs >= band[0]) & (freqs <= band[1])
    return np.sum(fftv[mask])

def extract_features(trials, fs_list):
    bands = {'theta': (4, 7), 'alpha': (8, 12), 'beta': (13, 30), 'gamma': (30, 50)} # Each signal converted to values of these 4 parameters
    feats = []
    for i, sig in enumerate(trials):
        fs = fs_list[i]
        sig_f = bandpass_iir(sig, fs)
        feat = [compute_band_power(sig_f, fs, b) for b in bands.values()]
        feats.append(feat)
    return np.array(feats)

In [ ]:
features = extract_features(eeg_trials, fs_list)
print("✅ Features shape:", features.shape)

scaler = StandardScaler()
X = scaler.fit_transform(features)
y = np.array(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

clf = GradientBoostingClassifier(n_estimators=200, random_state=42)  # 200 Iterations
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n🎯 Accuracy: {acc:.2f}")

print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\n📝 Classification Report:\n", classification_report(y_test, y_pred))

✅ Features shape: (1260, 4)

🎯 Accuracy: 0.85

📊 Confusion Matrix:
 [[  9  27]
 [ 10 206]]

📝 Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.25      0.33        36
           1       0.88      0.95      0.92       216

    accuracy                           0.85       252
   macro avg       0.68      0.60      0.62       252
weighted avg       0.83      0.85      0.83       252

